In [1]:
import os
import cv2
from iptcinfo3 import IPTCInfo
import numpy as np
from random import sample 
import matplotlib.pyplot as plt
import shutil
import pickle
import pandas as pd

In [2]:
pw_path = '/mnt/c/Users/xpn381/Documents/data/lpw.txt'
file = open(pw_path, mode='r')
pw = file.read()

In [3]:
!echo $pw|sudo -S sudo -S mount -t drvfs d: /mnt/f

[sudo] password for xpn381: mount: /mnt/f: d: already mounted on /mnt/f.


In [4]:
sample_dir = "/mnt/f/OctoberJohanArchiveSample5k"
df_loc_csv = "/mnt/f/df5k.csv"
df_loc_xlsx = "/mnt/f/df5k.xlsx"

In [5]:
pkl_file = open('IPTC_keys.pkl', 'rb')

IPTC_keys = pickle.load(pkl_file)

pkl_file.close()

In [6]:
def image_check(img_path):
    
    """Only for testing! open and plots an image"""

    img_array = cv2.imread(img_path)
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

    plt.imshow(img_array)

    plt.show()

In [7]:
def get_images_path(sample_dir):
    
    """Creates a list of all paths in flatten dir"""
    
    path_list = []
    ID_list = []
    No_list = []
    
    for root, dirs, files in os.walk(sample_dir):
        for name in files:
            path = os.path.join(root, name)
            file_name = name.split('.')[0]
            file_number = int(file_name.split('S')[1])

            path_list.append(path)
            ID_list.append(file_name)
            No_list.append(file_number)
            
    return(path_list, ID_list, No_list)

Should also get lists of IPTC data to put into df now.. Date, Loaction, Publication status ect. Dict or list? Remeber to handle missing

In [8]:
# with count - which should be object detection... labelimg
# features = ['ID','path','infants','children','youth','adults',
#             'elderly','female','male','no_scarf','Shayla_Dupatta','Hijab_Khimar_Chador',
#             'Niqab','Burqa','praying_person', 'Salah', 'protest','military_personnel',
#             'firearms','military_vehicles','US_flag','irq_flag','other_flag','cuffed_person',
#             'blindfolded_person', 'bound_person', 'barred_person','Casualties','blood','home',
#             'public_space','on_the_road','military_base','aerial','unidentified_scene']

# without count
features =  ['img_path','ID','number','salah', 'protest','blood','homely_scene',
            'public_space_scene','on_the_road_scene','military_base_scene','aerial_scene','unidentified_scene']

In [9]:
path_list5k, ID_list5k, No_list5k = get_images_path(sample_dir)

In [10]:
index = np.arange(len(No_list5k))

df = pd.DataFrame(index = index, columns = features)
df['ID'] = ID_list5k
df['img_path'] = path_list5k
df['number'] = No_list5k
df.fillna('0', inplace = True )
df.sort_values('number', ascending=True, inplace = True)
df.reset_index(drop=True, inplace=True)

In [11]:
df.head()

,img_path,ID,number,salah,protest,blood,homely_scene,public_space_scene,on_the_road_scene,military_base_scene,aerial_scene,unidentified_scene
0,/mnt/f/OctoberJohanArchiveSample5k/JS7.jpg,JS7,7,0,0,0,0,0,0,0,0,0
1,/mnt/f/OctoberJohanArchiveSample5k/JS9.jpg,JS9,9,0,0,0,0,0,0,0,0,0
2,/mnt/f/OctoberJohanArchiveSample5k/JS52.jpg,JS52,52,0,0,0,0,0,0,0,0,0
3,/mnt/f/OctoberJohanArchiveSample5k/JS53.jpg,JS53,53,0,0,0,0,0,0,0,0,0
4,/mnt/f/OctoberJohanArchiveSample5k/JS67.jpg,JS67,67,0,0,0,0,0,0,0,0,0


In [12]:
df.shape

(5000, 12)

In [13]:
df.to_csv(df_loc_csv, sep = ',')
df.to_excel(df_loc_xlsx)

# Testing

In [ ]:
df2=pd.read_excel(df_loc_xlsx, index_col = 0)

In [ ]:
df2.head()

In [27]:
for i in path_list5k[0:5]:
    dst = sample_dir + '/' + i.split('/')[-1]
    
    testinfo = IPTCInfo(dst, force = True)
    print(dst)
        
    for i in IPTC_keys:
        if testinfo[i]:
            print(f'key: {i}, info: {testinfo[i]}\n')
    
    image_check(dst)